In [ ]:
import mediapipe as mp
import os
import cv2

sample_size = 500
#print(os.path.abspath(".."))

path_to_hands = "../Hands/"
image_names = os.listdir(path_to_hands)[:sample_size]
fns = [os.path.splitext(fn)[0] for fn in image_names]



def get_hand_bbs(frame, hands, padding=20, draw_on_frame=True):
    """
        Given a frame, detects hand in the image and returns their bounding boxes in image space
        Can add padding onto the bounding boxes if needed.
        Can disable drawing on the frame if needed.
    """

    frame.flags.writeable = False
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame)
    frame.flags.writeable = True

    bboxes = []

    # Draw the hand annotations on the frame.
    if results.multi_hand_landmarks:

        for myHand in results.multi_hand_landmarks:
            xList = []
            yList = []
            for id, lm in enumerate(myHand.landmark):
                xList.append(lm.x)
                yList.append(lm.y)

            h, w, c = frame.shape
            xmin, xmax = max(int(min(xList) * w) - padding,
                             0), min(int(max(xList) * w) + padding, w)

            ymin, ymax = max(int(min(yList) * h) - padding,
                             0), min(int(max(yList) * h) + padding, h)

            # bboxInfo = {"id": id, "bbox": bbox,"center": (cx, cy)}
            bboxes.append(((xmin, ymin), (xmax, ymax)))

            if draw_on_frame:
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0),
                              2)

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return frame, bboxes

#sift = cv2.SIFT_create()

#pool = mp.Pool(processes=4)

mp_hands = mp.solutions.hands


gray_scale_images=[]
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    static_image_mode=True,
    max_num_hands=1,
)as hands:
    for img_path in tqdm(image_names):
        
